In [1]:
import tensorflow as tf
import os
import cv2
import numpy as np
import random as rnd

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

In [2]:
image_dim = 64

In [3]:
original = np.load(r'data/images/augmented/original.npy', allow_pickle=True)

original_images = []
original_labels = []

for idx, d in enumerate(original):
    original_images.extend(d)
    for e in d:
        original_labels.append([idx])

original_images = np.array(original_images)
original_labels = np.array(original_labels)

p = np.random.permutation(len(original_images))
original_images = original_images[p]
original_labels = original_labels[p]

In [4]:
dataset = np.load(r'data/images/augmented/augmentation.npy', allow_pickle=True)

images = []
labels = []

for idx, d in enumerate(dataset):
    images.extend(d)
    for e in d:
        labels.append([idx])

In [5]:
images = np.array(images)
labels = np.array(labels)

In [6]:
p = np.random.permutation(len(images))
images = images[p]
labels = labels[p]

In [7]:
ratio = 1

train_images, train_labels = images[:int(len(images)*ratio)], labels[:int(len(images)*ratio)]
test_images, test_labels = images[int(len(images)*ratio):], labels[int(len(images)*ratio):]

In [8]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(image_dim, image_dim, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (2, 2), activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(4))

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 13, 128)       32896     
_________________________________________________________________
flatten (Flatten)            (None, 21632)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               2

In [10]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=8, batch_size=32, shuffle=True, use_multiprocessing=False,
                    validation_data=(original_images, original_labels), validation_split=0.5)

Epoch 1/8
12499/12499 [==============================] - 654s 52ms/step - loss: 0.9042 - accuracy: 0.6964 - val_loss: 0.5956 - val_accuracy: 0.7792
Epoch 2/8
 1153/12499 [=>............................] - ETA: 7:52 - loss: 0.6014 - accuracy: 0.7762

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)

In [ ]:
print(test_acc)

In [ ]:
class_names = ['Bottle opener', 'Can opener', 'Cork screw', 'Multitool']
start = rnd.randint(0, len(test_images) - 25)

preds = model.predict(test_images[start:start+25])
index_max = []
for pred in preds:
    index_max.append(np.argmax(pred))

fig = plt.figure(figsize=(10, 10))
fig.patch.set_facecolor('xkcd:mint green')
for i in range(25):
    plt.subplot(5, 5, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(test_images[i+start])
    plt.tight_layout()
    plt.xlabel(
        'Actual: ' + class_names[test_labels[i+start][0]] +
        '\nPredicted: ' + class_names[index_max[i]])

In [ ]:
#model.save('models/v2')

In [ ]:
#model = tf.keras.models.load_model('models/v2')

In [ ]:
test_loss, test_acc = model.evaluate(original_images, original_labels, verbose=1)

In [ ]:
class_names = ['Bottle opener', 'Can opener', 'Cork screw', 'Multitool']
start = rnd.randint(0, len(original_images) - 25)

preds = model.predict(original_images[start:start+25])
index_max = []
for pred in preds:
    index_max.append(np.argmax(pred))

fig = plt.figure(figsize=(10, 10))
fig.patch.set_facecolor('xkcd:mint green')
for i in range(25):
    plt.subplot(5, 5, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(original_images[i+start])
    plt.tight_layout()
    plt.xlabel(
        'Actual: ' + class_names[original_labels[i+start][0]] +
        '\nPredicted: ' + class_names[index_max[i]])

In [ ]:
from DataAugmentation import resizeAndPad, rotations
import cv2

class_names = ['Bottle opener', 'Can opener', 'Cork screw', 'Multitool']
file_actual = cv2.imread(r'data/test/test11.jpg', cv2.COLOR_BGR2RGB)
files = rotations(resizeAndPad(file_actual, (image_dim, image_dim)), 100)

prediction = model.predict(np.array(files))
index_max = np.bincount(np.array([np.argmax(pred) for pred in prediction])).argmax()

plt.figure(figsize=(10, 10))
plt.grid(False)
plt.subplot(2, 2, 1)
plt.imshow(file_actual)
plt.subplot(2, 2, 2)
plt.imshow(files[0])

print ('Is your specified object a "{}"?'.format(class_names[index_max]))

In [ ]:
prediction

In [ ]:
index_max